<center><h1>TP4 RCR2:</h1></center>
<center><h2>logique possibiliste qualitative</h2></center>

## 0) Importation des librairies nécessaire

In [31]:
import random
import os
import pycosat

## 1) Création de notre base de connaissance

In [ ]:
class BaseConnaissance:
    number_of_clause   = 0
    number_of_variable = 0
    evidence_length = 0
    var_list = []
    max_clause_length = 0

    def __init__(self,nb_clause,nb_variable,length,max_number):
        self.number_of_clause = nb_clause
        self.number_of_variable = nb_variable
        self.evidence_length = length
        self.max_clause_length = max_number
        for i in range(self.number_of_variable):
            self.var_list.append(i+1)
            self.var_list.append(-(i+1))

    def get_evidence(self):
        return random.sample(self.var_list, self.evidence_length)

    def get_sigma(self):
        os.system("touch temp.txt")
        sigma_file = open("temp.txt","w")
        for i in range(self.number_of_clause):
            ran = random.randrange(1,self.max_clause_length + 1,1)
            temp_list = random.sample(self.var_list, ran)
            weight = random.random()
            weight = float(int(weight*100))/100
            chaine = "" + str(weight)+ " "
            for i in range(len(temp_list)):
                chaine = chaine + str(temp_list[i]) + " "
            sigma_file.write(chaine + "\n")
            temp_list=[]
        sigma_file.close()

## 2) Traitement de notre base de connaissance

In [32]:
class Sigma:
    lower = 0
    upper = 0
    weights = []
    formulas = []
    strates_weights = []
    def __init__(self,path_file): 
        file = open(path_file)
        for line in file:
            information = line.split()
            size = len(information)
            self.weights.append(float(information[0]))
            self.formulas.append([int(information[i]) for i in range(size) if i != 0]) 
        self.length = len(self.weights)
        file.close()

    def get_length(self):
        return self.length

    def get_weights(self):
        return self.weights

    def get_formulas(self):
        return self.formulas

    def sort_weights(self): 
        for i in range(len(self.weights)-1, 0, -1):
            for j in range(i):
                if self.weights[j] < self.weights[j+1]:
                    self.weights[j], self.weights[j+1] = self.weights[j+1], self.weights[j]
                    self.formulas[j], self.formulas[j+1] = self.formulas[j+1], self.formulas[j]

    def compute_strates(self): 
        no_doubles = set(self.weights)
        self.upper = len(no_doubles)
        self.strates_weights = sorted(no_doubles)

    def get_strates_number(self):
        return self.upper

    def get_strates_weights(self):
        return self.strates_weights

    def get_preprocessed_formulas(self,sub_formulas):
        dict_cor={}
        returned_formulas=[]
        cpt=1
        i=0
        for form in sub_formulas:
            j=0
            returned_formulas.insert(i,[])
            for pred in form:
                if(pred not in dict_cor.keys()):
                    if(int(pred)>0):
                        dict_cor[int(pred)]=cpt
                        dict_cor[-int(pred)]=-cpt
                        cpt+=1
                    else:
                        dict_cor[-int(pred)]=cpt
                        dict_cor[int(pred)]=-cpt
                        cpt+=1
                returned_formulas[i].insert(j,dict_cor[pred])
                j+=1
            i+=1
        return returned_formulas


## 3) Création et exécution de notre algorithme d'inférence

In [34]:
choice = 1 
if choice == 1 :
    base_de_connaissance = Sigma("file.wcnf") #Specification of file
    evidence = [1,-2,-3]
else :
    nb_clause   = 8
    nb_variable = 5
    length    = 3
    max_number = 4
    gen = BaseConnaissance(nb_clause,nb_variable,length,max_number)
    gen.get_sigma()
    base_de_connaissance = Sigma("temp.txt")
    evidence = gen.get_evidence()

base_de_connaissance.sort_weights()
base_de_connaissance.compute_strates() 
iteration = 1
while base_de_connaissance.lower < base_de_connaissance.upper:
    r =int( (base_de_connaissance.lower + base_de_connaissance.upper + 1)/2)
    
    liste = base_de_connaissance.get_weights()
    value_of_r =-1
    for i in range(len(liste)):
        if(base_de_connaissance.get_strates_weights()[r-1]>liste[i]):
            value_of_r = i
            break
    if value_of_r==-1:
        value_of_r = len(liste)-1
    
    for j in range(len(liste)):
        if base_de_connaissance.get_strates_weights()[base_de_connaissance.upper-1] == liste[j] :
            valueU = j
            break
    cnf = base_de_connaissance.formulas[valueU:value_of_r]
    
    for i in range(len(evidence)):
        liste = []
        liste.append(-1 * evidence[i])
        cnf.append(liste)
    cnf = base_de_connaissance.get_preprocessed_formulas(cnf)

    result = pycosat.solve(cnf)
    if type(result) == type([]): 
        base_de_connaissance.upper = r - 1

    else:
        base_de_connaissance.lower = r

    iteration = iteration + 1

Val = base_de_connaissance.get_strates_weights()[r-1]

f = open("file.wcnf")
content = f.read().splitlines()
print("L'exécution d'une base de connaissance qui contient : ")
for c in content:
    print(c)
print("Et avec une évidence de : ",evidence)
print("Donne le résultat suivant : Val("+str(evidence)+", Sigma) = "+str(Val))

L'exécution d'une base de connaissance qui contient : 
0.6 1 2
0.55 2 3
0.5 -2
0.4 4
0.3 5
0.2 6
Et avec une évidence de :  [1, -2, -3]
Donne le résultat suivant : Val([1, -2, -3], Sigma) = 0.5
